In [1]:
import folium
import requests
import json
import geopandas as gpd
from pprint import pprint

In [2]:
stations = gpd.read_file('./data/Wellington_New_Zealand/lds-nz-rail-station-points-topo-150k-SHP/nz-rail-station-points-topo-150k.shp')
districts = gpd.read_file('./data/Wellington_New_Zealand/wellington_area_pop.shp')

In [3]:
# districts.columns
districts['pop_dens'] = districts['VAR_1_3'] / districts['AREA_SQ_KM']
# mask = districts['SA22023__1'].apply(lambda x: 'wellington' in x or 'Wellington' in x)
# districts[mask].shape[0]

In [4]:
districts.to_crs(4326, inplace=True)
stations.to_crs(4326, inplace=True)

In [5]:
map = folium.Map(location=[-41.2865,174.7762], zoom_start=11, tiles='CartoDB Positron')
# folium.GeoJson(json.loads(result['isochrones'])).add_to(map)
folium.Choropleth(
    geo_data=districts['geometry'],  
    data=districts['pop_dens'], 
    columns=['geometry', 'pop_dens'], 
    key_on='feature.id', 
    fill_color='YlGnBu', 
    fill_opacity=0.7, 
    line_opacity=0.2, 
    legend_name="Population Density"
).add_to(map)
folium.GeoJson(stations).add_to(map)
map

In [8]:
filtered_stations = stations.clip(districts)
filtered_dists = districts.merge(districts.sjoin(filtered_stations).groupby('SA22023__1').size().rename('station_count').reset_index())
filtered_dists = filtered_dists.sort_values(by='station_count',ascending=False)[0:2]
filtered_stations = filtered_stations.clip(filtered_dists)

In [9]:
map = folium.Map(location=[-41.2865,174.7762], zoom_start=11, tiles='CartoDB Positron')
# folium.GeoJson(json.loads(result['isochrones'])).add_to(map)
folium.Choropleth(
    geo_data=filtered_dists['geometry'],  
    data=filtered_dists['station_count'], 
    columns=['geometry', 'station_count'], 
    key_on='feature.id', 
    fill_color='YlGnBu', 
    fill_opacity=0.7, 
    line_opacity=0.2, 
    legend_name="Station count"
).add_to(map)
folium.GeoJson(filtered_stations).add_to(map)
bounds = filtered_dists.total_bounds
map.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])
map

In [10]:
filtered_center_points = list(zip(*[filtered_stations.geometry.y, filtered_stations.geometry.x]))
filtered_center_points

[(-41.18529207595683, 174.8282774473257),
 (-41.17656439760665, 174.826921773871),
 (-41.17498785756021, 174.82715298120047),
 (-41.16929156693047, 174.82839241230963),
 (-41.14964287622999, 174.83815515549924),
 (-41.13761247104578, 174.8433887042614)]

In [11]:
# url = 'http://localhost:8086/isomap'
url = 'http://135.234.212.65:8086/isomap'
data = {
    'center_points': filtered_center_points, 
    'networktype': 'walk', 
    'travel_time': 15
}
payload = {
    "data": data, 
    "speeds": {
        "walk": 4500,
        "bike": 18000,
        "drive": 60000
    }
}

In [12]:
response = requests.post(url, json=payload)
if response.status_code == 200: 
    print("Request was successful. ")
    result = response.json()
else: 
    print(f"Failed to retreive response. Response message: {response.status_code}")

Request was successful. 


In [14]:
districts = districts.merge(districts.sjoin(filtered_stations, how='left').groupby('SA22023__1').size().rename('station_count').reset_index())

In [16]:
map = folium.Map(location=[38.895,-77.036389], zoom_start=13, tiles='CartoDB Positron')
folium.Choropleth(
    geo_data=districts['geometry'],  
    data=districts['station_count'], 
    columns=['geometry', 'station_couint'], 
    key_on='feature.id', 
    fill_color='YlGnBu', 
    fill_opacity=0.7, 
    line_opacity=0.2, 
    legend_name="Station Count"
).add_to(map)
folium.GeoJson(filtered_dists, 
              style_function=lambda feature: {
                  'fillColor': 'white', 
                  'color': 'black', 
                  'weight': 1.5,
                  'fillOpacity': 0
              }).add_to(map)
folium.GeoJson(json.loads(result['isochrones'])).add_to(map)
folium.GeoJson(filtered_stations).add_to(map)
bounds = filtered_dists.total_bounds
map.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])
map

In [17]:
isomap = gpd.read_file(result['isochrones'])
isomap.to_file('./results/isomap_rail_stations_2_SA2_highest_station_count_Wellington_NZ.shp')
map.save('./results/isomap_rail_stations_2_SA2_highest_station_count_Wellington_NZ.html')